Import libraries

In [ ]:
import pandas as pd
import os
import glob
from datetime import datetime

Filter data to get uniform format, in this case Solar generation until july 2024

In [ ]:
#read csv
df = pd.read_csv('/content/drive/MyDrive/Generation Solar 2024 nicht sortiert.csv', delimiter=',', quotechar='"')

#extract date
def extract_start_datetime(time_range):
    start_time_str = time_range.split(' - ')[0]
    return datetime.strptime(start_time_str, '%d.%m.%Y %H:%M')

#new date column
df['Start DateTime'] = df['MTU'].apply(extract_start_datetime)

#show until 01.08.2024
end_date = datetime(2024, 8, 1)
filtered_df = df[df['Start DateTime'] < end_date]

#remove columns
filtered_df = filtered_df[['Area', 'MTU', 'Solar  - Actual Aggregated [MW]']]

#save to csv
filtered_df.to_csv('/content/drive/MyDrive/Dataset Energiedaten/filtered_data.csv', index=False)

Import all csv from one folder, aggregate to a monthly value and create one csv (Solar generation)

In [ ]:
#path to csv
data_path = "/content/drive/MyDrive/Dataset Energiedaten"

#load all csv from one folder
csv_files = glob.glob(os.path.join(data_path, "*.csv"))

dataframes = []

#add csv to dataframe
for file in csv_files:
    df = pd.read_csv(file)

    #fit the date and clean the csv
    df['Datum'] = pd.to_datetime(df['MTU'].str.split(' - ').str[0], format='%d.%m.%Y %H:%M')
    monthly_df = df.groupby(df['Datum'].dt.to_period('M'))['Solar  - Actual Aggregated [MW]'].sum().reset_index()
    monthly_df['Datum'] = monthly_df['Datum'].dt.to_timestamp()
    monthly_df = monthly_df[['Datum', 'Solar  - Actual Aggregated [MW]']]
    monthly_df.columns = ['Datum', 'Werte (MW)']

    #add datafram to list
    dataframes.append(monthly_df)

#concat dataframes
final_df = pd.concat(dataframes)

#sort by date
final_df = final_df.sort_values(by='Datum').reset_index(drop=True)

#export csv
final_df.to_csv("/content/drive/MyDrive/Dataset Energiedaten/monthly_values.csv", index=False)

Filter data to get uniform format, in this case day ahead load until july 2024

In [ ]:
#read csv
df = pd.read_csv('/content/drive/MyDrive/not sortedLoad Day ahead 2024.csv', delimiter=',', quotechar='"')

df['Start DateTime'] = df['Time (CET/CEST)'].apply(extract_start_datetime) # Changed 'MTU' to 'Time (CET/CEST)'
df['Start DateTime'] = pd.to_datetime(df['Start DateTime'])

#set enddate to filter by (01.08.2024)
end_date = datetime(2024, 8, 1)

filtered_df = df[df['Start DateTime'] < end_date]

#save to csv
filtered_df.to_csv("/content/drive/MyDrive/Dataset dayahead load/filtered Load Day ahead 2024.csv", index=False)

Import all csv from one folder and create one csv (day ahead load)

In [ ]:
#path to csv
data_path = "/content/drive/MyDrive/Dataset dayahead load"

#load all csv in the folder
csv_files = glob.glob(os.path.join(data_path, "*.csv"))

dataframes = []

#add csv to dataframe
for file in csv_files:
    df = pd.read_csv(file)

    #fit the date and clean the csv
    df['Start DateTime'] = df['Time (CET/CEST)'].apply(extract_start_datetime)
    df['Start DateTime'] = pd.to_datetime(df['Start DateTime'])
    monthly_df = df.groupby(df['Start DateTime'].dt.to_period('M'))[
        ['Day-ahead Total Load Forecast [MW] - Germany (DE)', 'Actual Total Load [MW] - Germany (DE)']
    ].sum().reset_index()
    monthly_df['Start DateTime'] = monthly_df['Start DateTime'].dt.to_timestamp()
    monthly_df = monthly_df[['Start DateTime', 'Day-ahead Total Load Forecast [MW] - Germany (DE)', 'Actual Total Load [MW] - Germany (DE)']]

    #add dataframe to list
    dataframes.append(monthly_df)

# Dataframes zusammenfügen
final_df = pd.concat(dataframes)

#sort by date
final_df = final_df.sort_values(by='Start DateTime').reset_index(drop=True)

#save to csv
final_df.to_csv("/content/drive/MyDrive/Dataset dayahead load/monthly_day ahead loads.csv", index=False)

###Merge all csv to one csv

In [ ]:
#load day ahead load and solar generation
data1_day_ahead_load = pd.read_csv('/content/drive/MyDrive/DAI Projekt alle Date/monthly_day ahead loads.csv', delimiter=',', quotechar='"')
data2_solar_generation = pd.read_csv('/content/drive/MyDrive/DAI Projekt alle Date/Monthly_solar_generation.csv', delimiter=',', quotechar='"')

df1 = pd.DataFrame(data1_day_ahead_load)
df2 = pd.DataFrame(data2_solar_generation)

#Change 'Datum (MEZ)' to 'Start DateTime'
df1['Start DateTime'] = pd.to_datetime(df1['Start DateTime'])
df2['Datum'] = pd.to_datetime(df2['Datum'])

#merge day ahead load and solar generation on date
merged_df = pd.merge(df1, df2, left_on='Start DateTime', right_on='Datum', how='outer')
merged_df = merged_df.drop(columns=['Datum'])
merged_df.to_csv('/content/drive/MyDrive/merged_df.csv', index=False)

Change the format of the solar hours csv to fit the others

In [ ]:
data3_solar_hours = pd.read_csv('/content/drive/MyDrive/DAI Projekt alle Date/Sonnenstunden.csv', delimiter=',', quotechar='"')

data3_solar_hours = pd.DataFrame(data3_solar_hours.iloc[:, [0, 1, 18]])

data = []

for jahr in range(2015, 2025):
    for monat in range(1, 13):
        #create date string in the right format
        datum = datetime(jahr, monat, 1).strftime('%d.%m.%Y')

        #filter data
        wert = data3_solar_hours[(data3_solar_hours['Jahr'] == jahr) & (data3_solar_hours['Monat'] == monat)]['Deutschland'].values

        if wert.size > 0:
            wert = wert[0]
        else:
            wert = None

        data.append([datum, wert])

data3_solar_hours = pd.DataFrame(data, columns=['Datum', 'Wert'])

Change the format of the electricity prices to fith the others

In [ ]:
data6_electricity_prices = pd.read_csv('/content/drive/MyDrive/DAI Projekt alle Date/electricity_prices.csv')

#Change the format string to match the actual date format
data6_electricity_prices['Date'] = pd.to_datetime(data6_electricity_prices['Date'], format='%Y-%m-%d')

#filter the columns by the first day of the month
data6_electricity_prices = data6_electricity_prices[data6_electricity_prices['Date'].dt.day == 1]

data6_electricity_prices = pd.DataFrame(data6_electricity_prices.iloc[:, [5, 6]])
data6_electricity_prices['ID'] = range(1, len(data6_electricity_prices) + 1)

## GPR Index clearing data

In [ ]:
gpr_index = pd.read_excel('/content/drive/MyDrive/data_gpr_export.xls')

gpr_index['month'] = pd.to_datetime(gpr_index['month'])

# Filtern der Daten ab 2015
gpr_index = gpr_index[gpr_index['month'] >= '2015-01-01']

gpr_index = gpr_index[['month', 'GPRHC_DEU', 'GPRHC_RUS', 'GPRHC_UKR', 'GPRHC_USA']]

gpr_index['ID'] = range(1, len(gpr_index) + 1)

print(gpr_index.head())

          month  GPRHC_DEU  GPRHC_RUS  GPRHC_UKR  GPRHC_USA  ID
1380 2015-01-01   0.503291   0.645245   0.374242   3.187508   1
1381 2015-02-01   0.769661   1.287434   0.979569   3.372516   2
1382 2015-03-01   0.705253   0.887646   0.303988   3.343872   3
1383 2015-04-01   0.428319   0.831444   0.214160   3.174603   4
1384 2015-05-01   0.186474   0.459970   0.186474   2.299851   5


Continue merging the csv

In [ ]:
#adding ID's to simplify the merge process
data3_solar_hours['ID'] = range(1, len(data3_solar_hours) + 1)
merged_df['ID'] = range(1, len(merged_df) + 1)

final_merged_df = pd.merge(merged_df, data3_solar_hours, left_on='ID', right_on='ID', how='outer')

data4_day_ahead_solar_generation = pd.read_csv('/content/drive/MyDrive/DAI Projekt alle Date/Data_Ahead_solar_bereinigt.csv')
data4_day_ahead_solar_generation['ID'] = range(0, len(data4_day_ahead_solar_generation))
final_merge_df2 = pd.merge(final_merged_df, data4_day_ahead_solar_generation, left_on='ID', right_on='ID', how='outer')


data5_selected_data_solar_diff = pd.read_csv('/content/drive/MyDrive/DAI Projekt alle Date/selected_data_solar_diff.csv')
data5_selected_data_solar_diff['ID'] = range(0, len(data5_selected_data_solar_diff))

#add to dataframe
final_merge_df3 = pd.merge(final_merge_df2, data5_selected_data_solar_diff, left_on='ID', right_on='ID', how='outer')
final_merge_df4 = pd.merge(final_merge_df3, data6_electricity_prices, left_on='ID', right_on='ID', how='outer')

final_merge_df5 = pd.merge(final_merge_df4, gpr_index, left_on='ID', right_on='ID', how='outer')


#----------------------------clean up the final dataframe-------------------------------------------------------------------

#rename date column
final_merge_df5['Date'] = final_merge_df5['Start DateTime'].dt.strftime('%B %Y')

#remove unneccesary columns
final_merge_df5.drop(columns=['Start DateTime','ID', 'Datum', 'Datum (MEZ)_x', 'Datum (MEZ)_y','Solar','month'], inplace=True)

#move date to the first column
final_merge_df5 = final_merge_df5[[final_merge_df5.columns[-1]] + [col for col in final_merge_df5.columns if col != final_merge_df5.columns[-1]]]

#rename columns
final_merge_df5.rename(columns={'Wert': 'Solar hours (h)'}, inplace=True)
final_merge_df5.rename(columns={'Werte (MW)': 'Solar Generation (MW)'}, inplace=True)
final_merge_df5.rename(columns={'Day-ahead Total Load Forecast [MW] - Germany (DE)': 'Day-ahead Total Load Forecast [MW]'}, inplace=True)
final_merge_df5.rename(columns={'Actual Total Load [MW] - Germany (DE)': 'Actual Total Load [MW]'}, inplace=True)

# Konvertiere das 'Date' in ein Datetime-Format
final_merge_df5['Date'] = pd.to_datetime(final_merge_df5['Date'], format='%B %Y')

# Filtere die Daten bis zum 1. Juli 2024
final_merge_df5 = final_merge_df5[final_merge_df5['Date'] <= '2024-07-01']

final_merge_df5['Date'] = final_merge_df5['Date'].dt.strftime('%B %Y')

#export final csv
final_merge_df5.to_csv('/content/drive/MyDrive/Daten komplett.csv', index=False)